In [ ]:
import torch
from torch import nn
from torchrl.data import BoundedTensorSpec
from torchrl.modules.distributions.continuous import NormalParamWrapper, TanhNormal
from torchrl.modules.tensordict_module.actors import ProbabilisticActor, ValueOperator
from torchrl.modules.tensordict_module.common import SafeModule
from torchrl.objectives.sac import SACLoss
from tensordict import TensorDict
from torchrl.objectives.sac import SACLoss
from tensordict import TensorDict

In [ ]:
n_act, n_obs = 15,512
spec = BoundedTensorSpec(-torch.ones(n_act), torch.ones(n_act), (n_act,))

net = NormalParamWrapper(nn.Linear(n_obs, 2 * n_act))
module = SafeModule(net, in_keys=["observation"], out_keys=["loc", "scale"])
actor = ProbabilisticActor(
    module=module,
    in_keys=["loc", "scale"],
    spec=spec,
    distribution_class=TanhNormal)

In [ ]:
class ValueClass(nn.Module):
    def __init__(self,):
        super().__init__()
        self.linear = nn.Linear(n_obs + n_act, 1)
    def forward(self, obs, act):
        return self.linear(torch.cat([obs, act], -1))
    
module = ValueClass()

qvalue = ValueOperator(
    module=module,
    in_keys=['observation', 'action'])

loss = SACLoss(actor, qvalue)

In [ ]:
# batch = [2, ]
# action = spec.rand(batch)
# buffer_lazytensor = ReplayBuffer(storage=LazyTensorStorage(size))
# from torchrl.envs.transforms import CatTensors

In [1]:
import torch
from tensordict import TensorDict
from torchrl.data import ReplayBuffer
from torchrl.data.replay_buffers.samplers import PrioritizedSampler
from torchrl.data import LazyMemmapStorage, LazyTensorStorage, ListStorage
from torchrl.data import TensorDictReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
size = 100

In [ ]:
from torchrl.envs.transforms import (
    Compose,
    GrayScale,
    Resize,
    ToTensorImage,
    TransformedEnv,
)
a =  Compose(
        ToTensorImage(in_keys=["pixels"], out_keys=["pixels_trsf"]),
        Resize(in_keys=["pixels_trsf"], w=64, h=64),
        GrayScale(in_keys=["pixels_trsf"]),
    )

In [2]:
from envs.photo_env import PhotoEnhancementEnv

Encoding testing data ...


  0%|          | 0/4 [00:00<?, ?it/s]

finished...
Encoding training data ...


  0%|          | 0/36 [00:00<?, ?it/s]

finished...


In [3]:
import torch
env = PhotoEnhancementEnv(64)
state = env.reset()
action = torch.rand(64,15)
next_state,rewards, dones  = env.step(action)

In [ ]:
from torchrl.envs.transforms import (
    Compose,
    GrayScale,
    Resize,
    ToTensorImage,
    TransformedEnv,CatTensors
)
transform = Compose(
    CatTensors(in_keys = [("state","encoded_enhanced_image"),("state","encoded_source")], out_key = ('state','encoded_observation')),
    CatTensors(in_keys = [("next_state","encoded_enhanced_image"),("next_state","encoded_source")], out_key = ('next_state','encoded_observation'))
)
transform = Compose(
    CatTensors(in_keys = ["encoded_enhanced_image","encoded_source"], out_key = 'encoded_observation'),
)

In [4]:
buffer_ = TensorDictReplayBuffer(
    storage=LazyMemmapStorage(size,), sampler=SamplerWithoutReplacement()
)

In [5]:
transition ={
    'state':state,
    'action':action,
    'next_state':next_state,
    'rewards':rewards,
    'done':dones,
}

In [19]:
1-torch.Tensor([True])

tensor([0.])

In [29]:
torch.Tensor(True)

TypeError: new(): data must be a sequence (got bool)

In [32]:
a= dones.clone()

In [33]:
a[0]=True

In [35]:
a.to(torch.float32)


tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [27]:
dones.to(torch.float32)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [9]:
td = TensorDict(transition ,batch_size=[state.shape[0]])

In [10]:
buffer_.extend(td)

tensor([28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
        64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
        82, 83, 84, 85, 86, 87, 88, 89, 90, 91])

In [ ]:
buffer_.device='cuda'

In [11]:
buffer_

TensorDictReplayBuffer(
    storage=LazyMemmapStorage(
        data=TensorDict(
            fields={
                action: MemoryMappedTensor(shape=torch.Size([100, 15]), device=cpu, dtype=torch.float32, is_shared=False),
                done: MemoryMappedTensor(shape=torch.Size([100]), device=cpu, dtype=torch.bool, is_shared=False),
                index: MemoryMappedTensor(shape=torch.Size([100]), device=cpu, dtype=torch.int64, is_shared=False),
                next_state: MemoryMappedTensor(shape=torch.Size([100, 1024]), device=cpu, dtype=torch.float32, is_shared=False),
                rewards: MemoryMappedTensor(shape=torch.Size([100]), device=cpu, dtype=torch.float32, is_shared=False),
                state: MemoryMappedTensor(shape=torch.Size([100, 1024]), device=cpu, dtype=torch.float32, is_shared=False)},
            batch_size=torch.Size([100]),
            device=cpu,
            is_shared=False), 
        shape=torch.Size([100]), 
        len=100, 
        max_size=100), 

In [13]:
buffer_.sample(12).to('cuda')

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([12, 15]), device=cuda:0, dtype=torch.float32, is_shared=True),
        done: Tensor(shape=torch.Size([12]), device=cuda:0, dtype=torch.bool, is_shared=True),
        index: Tensor(shape=torch.Size([12]), device=cuda:0, dtype=torch.int64, is_shared=True),
        next_state: Tensor(shape=torch.Size([12, 1024]), device=cuda:0, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([12]), device=cuda:0, dtype=torch.float32, is_shared=True),
        state: Tensor(shape=torch.Size([12, 1024]), device=cuda:0, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([12]),
    device=cuda,
    is_shared=True)